<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HANDWRITTEN DIGITS RECOGNITION USING CNN
---

Handwritten digits classification using **CNN**. This solution is implemented in `Julia` using the `Flux.jl` library

In [1]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


In [2]:
;wget -O Project.toml https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml

--2025-10-17 21:07:49--  https://raw.githubusercontent.com/a-mhamdi/jlai/refs/heads/main/Codes/Julia/Part-3/cnn/Project.toml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 707 [text/plain]
Saving to: ‘Project.toml’

     0K                                                       100% 66.5M=0s

2025-10-17 21:07:49 (66.5 MB/s) - ‘Project.toml’ saved [707/707]



In [3]:
;sed -i -e '/^Pl[uo]/d' -e '/^Image/d' Project.toml

In [4]:
import Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.update()

  Activating project at `/content`
┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API /usr/local/share/julia/stdlib/v1.11/Pkg/src/API.jl:1206
Precompiling project...
┌ Warning: attempting to remove probably stale pidfile
│   path = "/root/.julia/compiled/v1.11/CUDA/oWw5k_MJWkB.ji.pidfile"
└ @ FileWatching.Pidfile /usr/local/share/julia/stdlib/v1.11/FileWatching/src/pidfile.jl:249
   3725.6 ms  ✓ Transducers → TransducersDataFramesExt
   4181.0 ms  ✓ BangBang → BangBangDataFramesExt
  60506.7 ms  ✓ CUDA
┌ Warning: attempting to remove probably stale pidfile
│   path = "/root/.julia/compiled/v1.11/SpecialFunctionsExt/9h4VD_MJWkB.ji.pidfile"
└ @ FileWatching.Pidfile /usr/local/share/julia/stdlib/v1.11/FileWatching/src/pidfile.jl:249
┌ Warning: attempting to remove probably stale pidfile
│   path = "/root/.julia/compiled/v1.11/ChainRulesCore

In [5]:
import Pkg; Pkg.add(["CUDA", "cuDNN"]);

   Resolving package versions...
    Updating `/content/Project.toml`
⌅ [052768ef] + CUDA v5.8.5
  [02a925ec] + cuDNN v1.4.5
    Updating `/content/Manifest.toml`
  [fa961155] + CEnum v0.5.0
⌅ [052768ef] + CUDA v5.8.5
  [1af6417a] + CUDA_Runtime_Discovery v1.0.0
  [e2ba6199] + ExprTools v0.1.10
  [0c68f7d7] + GPUArrays v11.2.6
  [61eb1bfa] + GPUCompiler v1.7.2
⌅ [096a3bc2] + GPUToolbox v0.3.0
  [929cbde3] + LLVM v9.4.4
  [8b046642] + LLVMLoopInfo v1.0.0
  [5da4648a] + NVTX v1.0.1
⌅ [08abe8d2] ↓ PrettyTables v3.1.0 ⇒ v2.4.0
  [74087812] + Random123 v1.7.1
  [e6cf234a] + RandomNumbers v1.6.0
  [e689c965] + Tracy v0.1.6
  [02a925ec] + cuDNN v1.4.5
  [d1e2174e] + CUDA_Compiler_jll v0.2.2+0
  [4ee394cb] + CUDA_Driver_jll v13.0.2+0
  [76a88914] + CUDA_Runtime_jll v0.19.2+0
  [62b44479] + CUDNN_jll v9.13.0+0
  [9c1d0b0a] + JuliaNVTXCallbacks_jll v0.2.1+0
  [dad2f222] + LLVMExtra_jll v0.0.38+0
  [ad6e5548] + LibTracyClient_jll v0.9.1+6
  [e98f9f5b] + NVTX_jll v3.2.2+0
  [1e29f10c] + demumble_j

In [ ]:
import Pkg; Pkg.add(["ImageShow", "WGLMakie"]);

In [7]:
import Pkg; Pkg.precompile(); Pkg.resolve()

  No Changes to `/content/Project.toml`
  No Changes to `/content/Manifest.toml`


Import the machine learning library `Flux`

In [8]:
using Flux
using Flux: DataLoader
using Flux: onecold, onehotbatch

In [9]:
using CUDA
CUDA.versioninfo()

CUDA toolchain: 
- runtime 12.5, local installation
- driver 550.54.15 for 13.0
- compiler 12.9

CUDA libraries: 
- CUBLAS: 12.5.3
- CURAND: 10.3.6
- CUFFT: 11.2.3
- CUSOLVER: 11.6.3
- CUSPARSE: 12.5.1
- CUPTI: 2024.2.1 (API 12.5.0)
- NVML: 12.0.0+550.54.15

Julia packages: 
- CUDA: 5.8.5
- CUDA_Driver_jll: 13.0.2+0
- CUDA_Compiler_jll: 0.2.2+0
- CUDA_Runtime_jll: 0.19.2+0
- CUDA_Runtime_Discovery: 1.0.0

Toolchain:
- Julia: 1.11.5
- LLVM: 16.0.6

Preferences:
- CUDA_Runtime_jll.version: 12.5.1
- CUDA_Runtime_jll.local: true

1 device:
  0: Tesla T4 (sm_75, 14.738 GiB / 15.000 GiB available)


In [10]:
Base.@kwdef mutable struct HyperParams
    η = 3f-3                # Learning rate
    batchsize = 64          # Batch size
    epochs = 8              # Number of epochs
    split = :train          # Split data into `train` and `test`
end

HyperParams

Load the **MNIST** dataset

In [11]:
using MLDatasets

In [12]:
d = MNIST()

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :train
  features  =>    28×28×60000 Array{Float32, 3}
  targets   =>    60000-element Vector{Int64}

In [13]:
function get_data(; kws...)
    args = HyperParams(; kws...);
    # Split and normalize data
    data = MNIST(split=args.split);
    X, y = data.features ./ 255, data.targets;
    X = reshape(X, (28, 28, 1, :));
    y = onehotbatch(y, 0:9);
    loader = DataLoader((X, y); batchsize=args.batchsize, shuffle=true) |> gpu;
    return loader
end

get_data (generic function with 1 method)

In [14]:
train_loader = get_data();
test_loader = get_data(split=:test);

In [15]:
x1, y1 = first(get_data());
# CPU => Array...
# GPU => CuArray...
typeof((x1, y1))

Tuple{CuArray{Float32, 4, CUDA.DeviceMemory}, OneHotArrays.OneHotMatrix{UInt32, CuArray{UInt32, 1, CUDA.DeviceMemory}}}

Transform sample training data to an image. View the image and check the corresponding digit value.

In [16]:
using Statistics

In [17]:
idx = rand(1:6_000, 3)

3-element Vector{Int64}:
 5446
 2528
 1218

In [ ]:
using ImageShow

In [18]:
convert2image(d, idx)

LoadError: Add `import ImageShow` or `using ImageShow` to your code to unlock this functionality.

In [19]:
"Digit are $(d.targets[idx])"

"Digit are [4, 5, 7]"

**CNN** ARCHITECTURE

The input `X` is a batch of images with dimensions `(width=28, height=28, channels=1, batchsize)`

In [20]:
fc = prod(Int.(floor.([28/4 - 2, 28/4 - 2, 16]))) # 2^{\# max-pool}

400

In [21]:
model = Chain(
            Conv((5, 5), 1 => 16, relu),  # (28-5+1)x(28-5+1)x16 = 24x24x16
            MaxPool((2, 2)),              # 12x12x16
            Conv((3, 3), 16 => 16, relu), # (12-3+1)x(12-3+1)x16 = 10x10x16
            MaxPool((2, 2)),              # 5x5x16
            Flux.flatten,                 # 400
            Dense(fc => 64, relu),
            Dense(64 => 32, relu),
            Dense(32 => 10)
) |> gpu

Chain(
  Conv((5, 5), 1 => 16, relu),          # 416 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 16 => 16, relu),         # 2_320 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(400 => 64, relu),               # 25_664 parameters
  Dense(64 => 32, relu),                # 2_080 parameters
  Dense(32 => 10),                      # 330 parameters
)                   # Total: 10 arrays, 30_810 parameters, 1.859 KiB.

In [22]:
using ProgressMeter: Progress, next!

In [23]:
function train(; kws...)
    args = HyperParams(; kws...)
    # Define the loss function
    l(α, β) = Flux.logitcrossentropy(α, β)
    # Define the accuracy metric
    acc(α, β) = mean(onecold(α) .== onecold(β))
    # Optimizer
    optim_state = Flux.setup(Adam(args.η), model);

    vec_loss = []
    vec_acc = []

    for epoch in 1:args.epochs
        printstyled("\t***\t === EPOCH $(epoch) === \t*** \n", color=:magenta, bold=true)
        @info "TRAINING"
        prg_train = Progress(length(train_loader))
        for (X, y) in train_loader
            loss, grads = Flux.withgradient(model) do m
                ŷ = m(X);
                l(ŷ, y);
            end
            Flux.update!(optim_state, model, grads[1]); # Upd `W` and `b`
            # Show progress meter
            next!(prg_train, showvalues=[(:loss, loss)])
        end
        @info "TESTING"
        prg_test = Progress(length(test_loader))
        for (X, y) in test_loader
            ŷ = model(X);
            push!(vec_loss, l(ŷ, y));  # log `loss` value -> `vec_loss` vector
            push!(vec_acc, acc(ŷ, y)); # log `accuracy` value -> `vec_acc` vector
          	# Show progress meter
            next!(prg_test, showvalues=[(:loss, vec_loss[end]), (:accuracy, vec_acc[end])])
        end
    end
    return vec_loss, vec_acc
end

train (generic function with 1 method)

In [24]:
vec_loss, vec_acc = train()

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter ~/.julia/packages/ProgressMeter/N660J/src/ProgressMeter.jl:607
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
       loss: 2.2870932
   accuracy: 0.125


(Any[2.3016305f0, 2.2977686f0, 2.2997186f0, 2.296072f0, 2.3016214f0, 2.3024366f0, 2.3082395f0, 2.2978535f0, 2.3037832f0, 2.306711f0  …  2.300909f0, 2.3006914f0, 2.3036096f0, 2.3018723f0, 2.2952554f0, 2.2918506f0, 2.3001223f0, 2.2979794f0, 2.2966824f0, 2.2870932f0], Any[0.203125, 0.1875, 0.1875, 0.078125, 0.0625, 0.09375, 0.09375, 0.125, 0.140625, 0.078125  …  0.109375, 0.078125, 0.09375, 0.109375, 0.109375, 0.140625, 0.125, 0.125, 0.171875, 0.125])

Plot results

In [25]:
# Load the package
using WGLMakie

# Create data for sine wave
x = range(0, 4π, length=100)
y = sin.(x)

# Create the plot
fig = Figure(size = (800, 400))
ax1 = Axis(
    fig[1, 1],
    xlabel = "Iteration",
    title = "Test Loss vs. Iteration"
)
ax2 = Axis(
    fig[2, 1],
    xlabel = "Iteration",
    title = "Test Accuracy vs. Iteration"
)

# Plot the `vec_loss`
lines!(ax1, vec_loss, color = :red, linewidth = 2)

# Plot the `acc_loss`
lines!(ax2, acc_loss, color = :blue, linewidth = 2)

# Display the figure
fig

[ Info: Precompiling WGLMakie [276b4fcb-3e11-5398-bf8b-a0c2d153d008] (cache misses: wrong dep version loaded (2))

[16888] signal 2: Interrupt
in expression starting at /root/.julia/packages/WGLMakie/FbMOf/src/WGLMakie.jl:6
tname_intersection_dt at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/typemap.c:409 [inlined]
tname_intersection_dt at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/typemap.c:399 [inlined]
jl_typemap_intersection_memory_visitor at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/typemap.c:492
jl_typemap_intersection_visitor at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/typemap.c:733
jl_typemap_intersection_visitor at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/typemap.c:793
ml_mtable_visitor at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/gf.c:3347 [inlined]
ml_matches at /cache/build/tester-amdci5-12/julialang/julia-release-1-dot-11/src/

LoadError: Failed to precompile WGLMakie [276b4fcb-3e11-5398-bf8b-a0c2d153d008] to "/root/.julia/compiled/v1.11/WGLMakie/jl_6P8M9L".

Let's make some predictions

In [26]:
idx = rand(1:1000, 16)
xs = test_loader.data[idx][1]
yp = xs |> model |> softmax |> out -> onecold(out, 0:9) |> cpu
ys = onecold(test_loader.data[idx][2]) .- 1 |> cpu;

LoadError: type DeviceIterator has no field data

In [27]:
for i ∈ eachindex(yp)
    @info "**Prediction is $(yp[i]). Label is $(ys[i]).**"
end

LoadError: UndefVarError: `yp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Save and load the model

In [28]:
folder = "models"
isdir(folder) || mkdir(folder)

true

In [29]:
#=
using JLD2

filename_1 = joinpath(folder, "cnn.jld2")
JLD2.jldsave(filename; cnn_state = Flux.state(model) |> cpu)
Flux.loadmodel!(mdl_1, cnn_state)
=#

In [30]:
#=
using BSON: @save, @load

filename_2 = joinpath(folder, "cnn.bson")
@save filename_2 model
@load
=#